# Model_5dimensional
This file provides the code to trains a model to evaluate a relatively short text (20 words) to 5 index, neutral, happy, sad, anger, hate

### import packages

In [1]:
import warnings  
with warnings.catch_warnings():  
    warnings.filterwarnings("ignore",category=FutureWarning)
    import numpy as np
    import tensorflow as tf
    import tensorflow_datasets as tfds
    import matplotlib.pyplot as plt
    import numpy as np
    import os
    from tqdm import tqdm
    import pickle
    import csv
    import random
    import tensorflow as tf
    from tensorflow.keras import Sequential
    import numpy as np
    import matplotlib.pyplot as plt
    from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, Embedding, Reshape
print("Version: ", tf.__version__)

Version:  2.0.0-beta1


### Download the dataset and shuffle the texts 

In [2]:
url="https://raw.githubusercontent.com/tlkh/text-emotion-classification/master/data.csv"
import urllib.request
urllib.request.urlretrieve(url, "traindata.csv")
sentence=[]
sentiment=[]
texts=[]
with open("traindata.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        texts.append(row)
            
random.shuffle(texts)
print(texts[12])
for row in texts:
    sentence.append(row[0])
    sentiment.append(int(row[1]))
# neutral, happy, sad, anger, hate
print(len(sentence))

['i getvthat my dog seems to like carrot cake and devoured a slice i brought home from a resto last week angry i like cake', '4']
47288


### Fit the Tokenizer on texts and convert texts to sequences, divided data to training data and testing data

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=40000)
tokenizer.fit_on_texts(sentence)
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentence_token = tokenizer.texts_to_sequences(sentence)
train_x = pad_sequences(sentence_token[:40000], maxlen=20)
train_y= np.array(sentiment[:40000])
test_x = pad_sequences(sentence_token[40000:], maxlen=20)
test_y=np.array(sentiment[40000:])
print(train_x.shape)

(40000, 20)


### Feed the data to model and fit the data to model

In [4]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D
tf.keras.backend.clear_session()
model = Sequential()
model.add(Embedding(60000, 128))
model.add(tf.keras.layers.Conv1D(
                 100,
                 5,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='relu'))
model.add(Dense(5, activation='softmax'))
optim=tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.99, beta_2=0.999)
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optim,
              metrics=['accuracy'])
print(model.summary())
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(train_x, train_y , batch_size=32, epochs=12, validation_data=(test_x, test_y), callbacks=[callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         7680000   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 100)         64100     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
Total params: 7,786,695
Trainable params: 7,786,695
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4000

### Save the data and test on short texts

In [5]:
model.save('Model_5.h5', save_format='tf')

In [7]:
def process(text):
    text_p = tokenizer.texts_to_sequences([text])
    processed = pad_sequences(text_p, maxlen=20)
    return processed

In [8]:
process("Today is sunny")

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,  49,   9, 729]], dtype=int32)

In [9]:
np.argmax(model.predict(process("I am too tired if working")))

2